In [1]:
from absl import app, logging
from PIL import Image as PILImg
from PIL import ImageFilter
import PIL.ImageOps
from robokit.utils import annotate, overlay_masks
from robokit.ObjDetection import GroundingDINOObjectPredictor,SegmentAnythingPredictor
import torch 
gdino = GroundingDINOObjectPredictor(use_vitb=False, threshold=0.15)
SAM = SegmentAnythingPredictor(vit_model="vit_h")
# Path to the input image
image_path = 'D:/CODE/NIDS-Net/test_data/test_1/test_039.jpg'

final text_encoder_type: bert-base-uncased
Model loaded from C:\Users\divya\.cache\huggingface\hub\models--ShilongLiu--GroundingDINO\snapshots\a94c9b567a2a374598f05c584e96798a170c56fb\groundingdino_swint_ogc.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])


In [2]:
text_prompt = 'objects'

try:
    logging.info("Open the image and convert to RGB format")
    image_pil = PILImg.open(image_path).convert("RGB")
    
    logging.info("GDINO: Predict bounding boxes, phrases, and confidence scores")
    bboxes, phrases, gdino_conf = gdino.predict(image_pil, text_prompt)

    logging.info("GDINO post processing")
    w, h = image_pil.size # Get image width and height 
    # Scale bounding boxes to match the original image size
    image_pil_bboxes = gdino.bbox_to_scaled_xyxy(bboxes, w, h)  
    
    # logging.info("SAM: Predict")
    image_pil_bboxes, masks = SAM.predict(image_pil, image_pil_bboxes)
    print(masks.shape)
    
    # print(image_pil_bboxes)
    logging.info("Annotate the scaled image with bounding boxes, confidence scores, and labels, and display")
    bbox_annotated_pil = annotate(image_pil, image_pil_bboxes, gdino_conf, phrases)
    bbox_annotated_pil.show()
    del image_pil_bboxes
    torch.cuda.empty_cache()

except Exception as e:
    # Handle unexpected errors
    print(f"An unexpected error occurred: {e}")

INFO:absl:Open the image and convert to RGB format
INFO:absl:GDINO: Predict bounding boxes, phrases, and confidence scores
INFO:absl:GDINO post processing
INFO:absl:Annotate the scaled image with bounding boxes, confidence scores, and labels, and display


torch.Size([35, 1, 6144, 8192])


In [2]:
import torch
import torchvision.transforms as T
from PIL import Image
import matplotlib.pyplot as plt
from featup.util import norm, unnorm
from featup.util import pca, remove_axes

@torch.no_grad()
def plot_feats(image, lr, hr):
    assert len(image.shape) == len(lr.shape) == len(hr.shape) == 3
    [lr_feats_pca, hr_feats_pca], _ = pca([lr.unsqueeze(0), hr.unsqueeze(0)])
    fig, ax = plt.subplots(1, 3, figsize=(150, 50))
    ax[0].imshow(image.permute(1, 2, 0).detach().cpu())
    ax[0].set_title("Image")
    ax[1].imshow(lr_feats_pca[0].permute(1, 2, 0).detach().cpu())
    ax[1].set_title("Original Features")
    ax[2].imshow(hr_feats_pca[0].permute(1, 2, 0).detach().cpu())
    ax[2].set_title("Upsampled Features")
    plt.show()

image_path = 'D:/CODE/NIDS-Net/test_data/test_1/test_039.jpg'

input_size = 448
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
use_norm = True

transform = T.Compose([
    T.Resize(input_size),
    T.CenterCrop((input_size, input_size)),
    T.ToTensor(),
    norm
])

image_tensor = transform(Image.open(image_path).convert("RGB")).unsqueeze(0).to(device)

upsampler = torch.hub.load("mhamilton723/FeatUp", 'dino16', use_norm=use_norm).to(device)
hr_feats = upsampler(image_tensor)
lr_feats = upsampler.model(image_tensor)
plot_feats(unnorm(image_tensor)[0], lr_feats[0], hr_feats[0])

Using cache found in C:\Users\divya/.cache\torch\hub\mhamilton723_FeatUp_main
Using cache found in C:\Users\divya/.cache\torch\hub\facebookresearch_dino_main
